# Load Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

%matplotlib inline

# Train Decision Tree on IRIS to understand 

In [2]:
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target


tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [3]:
export_graphviz(
    tree_clf,
    out_file="iris_tree.dot",
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)
!dot -Tpng iris_tree.dot -o iris_tree.png

<img src="iris_tree.png" alt="Decision Tree"/>

<p class="alert alert-info">One of the many qualities of Decision Trees is that they require
very little data preparation. In particular, they don’t require feature
scaling or centering at all.</p>

# Let's breakdown how it works

<p style="color:gold">$$\Large J(k, t_{k}) = \frac{m_{\text{left}}}{m}G_{\text{left}} + \frac{m_{\text{right}}}{m}G_{\text{right}}$$</p>

$$ \text{where} \left\{ \begin{array}{rcl} G_{\text{left/right}}\; \text{measures the impurity of the left/right subset} \\ m_{\text{left/right}}\; \text{is the number of instances in the left/right subset} \end{array}\right.$$


<p style="color:white"> $$k = \text{feature}, \quad t_{k}=\text{threshold for feature}\; k$$</p>

$$ \text{Gini Impurity} (G) = 1 - \sum_{k=1}^{n}p_{i,k}^{2}$$

$p_{i,k}$ is the ratio of class k instances among the training instances in the ith node



In [4]:
initial_gini_index = 1
labels = np.unique(y)

for label in labels:
    initial_gini_index -= (y[y == label].shape[0] / y.shape[0])**2
    
print(f"Initial Gini Index even before splitting the dataset: {initial_gini_index:.3f}")

Initial Gini Index even before splitting the dataset: 0.667


## peal length (cm)

In [5]:
def get_feature_split(feature: np.ndarray, y: np.ndarray):
    
    min_gini = np.inf
    final_thresold = 0.0
    num_left = 0
    num_right = 0
    feat_col = np.column_stack((feature, y))
    feat_col = feat_col[feat_col[:, 0].argsort()]
    epsilon = 1e-6


    # caluclate each midpoint between each subsequent values which are our
    # potential thresholds
    pot_thresholds = (feat_col[:, 0][:-1] + feat_col[:, 0][1:]) / 2

    for th in pot_thresholds:

        # let's now get the left and right subset instances with it labels
        ls = feat_col[feat_col[:, 0] <= th]
        rs = feat_col[feat_col[:, 0] > th]

        # now take instance in left and right subsets and calculate how many instances
        # belong to which class to calculate the probablity for gini index
        g_left, g_right = 1, 1
        for label in labels:
            g_left -= ((ls[ls[:, 1] == label]).shape[0] / (ls.shape[0] + epsilon))**2
            g_right -= ((rs[rs[:, 1] == label]).shape[0] / (rs.shape[0] + epsilon))**2

        wg_l = (ls.shape[0] / feat_col.shape[0]) * g_left
        wg_r = (rs.shape[0] / feat_col.shape[0]) * g_right
        gini_weighted = wg_l + wg_r
        
        if gini_weighted <= min_gini:
            min_gini = gini_weighted
            final_thresold = th
            num_left = ls.shape[0]
            num_right = rs.shape[0]
            final_ls = ls
            final_rs = rs
    
        # print(f"Threshold: {th:.2f}, G_left: {g_left:.2f}, G_right: {g_right:.2f}, J: {gini_weighted:.2f}")
    
    return final_ls, final_rs, num_left, num_right, final_thresold, min_gini

# Test `Petal Length` feature for best split for initial node

In [6]:
ls, rs, num_left, num_right, final_thresold, min_gini = get_feature_split(X[:, 0], y)
print(f"\nNumber of Samples in Left Sub Node: {num_left}")
print(f"\nNumber of Samples in Right Sub Node: {num_right}")
print(f"\nSplit at threshold of column Petal Legnth(cm): {final_thresold:.2f} with Gini Weighted Split of {min_gini:.2f}")


Number of Samples in Left Sub Node: 50

Number of Samples in Right Sub Node: 100

Split at threshold of column Petal Legnth(cm): 2.45 with Gini Weighted Split of 0.33


# Test `Petal Width` feature for best split for initial node

In [7]:
ls, rs, num_left, num_right, final_thresold, min_gini = get_feature_split(X[:, 1], y)
print(f"\nNumber of Samples in Left Sub Node: {num_left}")
print(f"\nNumber of Samples in Right Sub Node: {num_right}")
print(f"Split at threshold of column Petal Legnth(cm): {final_thresold:.2f} with Gini Weighted Split of {min_gini:.2f}")


Number of Samples in Left Sub Node: 50

Number of Samples in Right Sub Node: 100
Split at threshold of column Petal Legnth(cm): 0.80 with Gini Weighted Split of 0.33


*Note:* Seems that we have same split for both of the features